In [73]:
import open3d as o3d
import numpy as np
from open3d.j_visualizer import JVisualizer

In [74]:
import os
import os.path

import numpy as np
from torchsparse import SparseTensor
from torchsparse.utils.collate import sparse_collate_fn
from torchsparse.utils.quantize import sparse_quantize

In [1]:
class ErasorCarlaInternal:

    def __init__(self,
                 root,
                 voxel_size,
                 num_points,
                 visualize,
                 split,
                 sample_stride=1,
                 submit=False,
                 google_mode=True,
                 window=10,
                 radius=50):
        if submit:
            trainval = True
        else:
            trainval = False
        self.root = root
        self.split = split
        self.voxel_size = voxel_size
        self.num_points = num_points
        self.sample_stride = sample_stride
        self.google_mode = google_mode
        self.window = window
        self.visualize = visualize
        self.radius = radius
        self.seqs = []
        if split == 'train':
            self.seqs = [
                'scenario2', 'scenario3', 'scenario5', 'scenario8',
            ]

        elif self.split == 'val':
            self.seqs = [
                'scenario6',
            ]
        elif self.split == 'test':
            self.seqs = [
               'scenario6',
            ]

        self.map_files = dict()
        self.files = []

        # get scan and map data list
        for seq in self.seqs:
            self.map_files[seq] = os.path.join(self.root, 'testing_map/v0.1', seq, 'map.npy')
            seq_files = sorted(os.listdir(os.path.join(self.root, 'testing_data', seq, 'global_npz')))
            seq_files = seq_files[(self.window + 1) // 2 - 1: - (self.window // 2) + 1]

            seq_files = [os.path.join(self.root, 'testing_data', seq, 'global_npz', x) for x in seq_files]
            self.files.extend(seq_files)

        # get map_data
        self.map = dict()
        for seq, map_file in self.map_files.items():
            map_ = np.load(map_file)
            # map = generate_voxels(map_, voxel_size=self.voxel_size)
            self.map[seq] = map_.astype(np.float32)

        if self.sample_stride > 1:
            self.files = self.files[::self.sample_stride]

        self.num_classes = 2
        self.angle = 0.0

    def set_angle(self, angle):
        self.angle = angle

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):

        # get scan_data
        scan = np.load(self.files[index])
        block_ = scan['arr_0'].astype(np.float32)
        odom = scan['arr_1'].astype(np.float32)

        if self.window > 1:
            file_name = os.path.basename(self.files[index])
            file_dir = os.path.dirname(self.files[index])
            file_name_wo_ext = os.path.splitext(file_name)[0]
            file_int = int(file_name_wo_ext)
            file_int_list = list(range(file_int - (self.window + 1) // 2 + 1, file_int + self.window // 2 + 1, 1))
            file_str_list = [file_dir + "/" + str(i).zfill(6) + ".npz" for i in file_int_list]
            block_ = []
            for file in file_str_list:
                scan_ = np.load(file)
                block_single = scan_['arr_0'].astype(np.float32)
                block_.extend(block_single)
            block_ = np.asarray(block_)
            # radius search w.r.t the odom of scan data
            block_ = block_[np.sum(np.square(block_[:, :3] - odom), axis=-1) < self.radius*self.radius]

        # get map_data
        scenario = self.files[index]
        scenario = scenario.split("/")[-3]
        map_ = self.map[scenario]
        # radius search w.r.t the odom of scan data
        map_ = map_[np.sum(np.square(map_[:, :3] - odom), axis=-1) < self.radius*self.radius]
        
        # get pesudo label with ERASOR
        """
        def get_pseudo_label(self, map_, block_, **kwargs):
            # return dynamic and static labels of map
            return map_[:, 3]
        pseudo_label = self.get_pseudo_label(map_, block_, **kwargs)
        """
        
        
        block = np.zeros_like(block_)
        map = np.zeros_like(map_)
        

        if 'train' in self.split:
            # data augmentation on the train dataset
            theta = np.random.uniform(0, 2 * np.pi)
            scale_factor = np.random.uniform(0.95, 1.05)
            rot_mat = np.array([[np.cos(theta), np.sin(theta), 0],
                                [-np.sin(theta),
                                 np.cos(theta), 0], [0, 0, 1]])

            block[:, :3] = np.dot(block_[:, :3], rot_mat) * scale_factor
            map[:, :3] = np.dot(map_[:, :3], rot_mat) * scale_factor

        else:
            theta = self.angle
            transform_mat = np.array([[np.cos(theta),
                                       np.sin(theta), 0],
                                      [-np.sin(theta),
                                       np.cos(theta), 0], [0, 0, 1]])
            block[...] = block_[...]
            block[:, :3] = np.dot(block[:, :3], transform_mat)
            map[:, :3] = np.dot(map_[:, :3], transform_mat)
            
#         # self-supervised dynamic label
#         center_x = np.mean(block[:, 0])
#         center_y = np.mean(block[:, 1])
#         sspoints_ = np.mgrid[center_x+2:center_x+3:self.voxel_size, center_y-2:center_y+3:self.voxel_size, 0:2:self.voxel_size]
#         sspoints = sspoints_.reshape(3, -1).T
#         sslabels = np.ones([np.shape(sspoints)[0], 1],)
#         ssblock = np.concatenate((sspoints, sslabels), axis=1)
        
#         block = np.concatenate((block, ssblock), axis=0)
        

        # parsing the original label to the dynamic label
        block[:, 3] = (block[:, 3] == 1)
        map[:, 3] = (map[:, 3] == 1)
        
#         # get point and voxel in the format of sparse torch tensor
#         map_data = self.get_point_voxel(map[:, :3], map[:, 3], index)
#         scan_data = self.get_point_voxel(block[:, :3], block[:, 3], index)
        
        # get original point and voxel
        map_data = self.get_original_point(map[:, :3], map[:, 3], index)
        scan_data = self.get_original_point(block[:, :3], block[:, 3], index)

#         return map_data
        return map_data, scan_data

    @staticmethod
    def collate_fn(inputs):
        return sparse_collate_fn(inputs)
    
    def get_original_point(self, block, labels_, index):
        pc_ = np.round(block[:, :3] / self.voxel_size).astype(np.int32)
        
#         # normalize the point for visualization
#         pc_ = (pc_ - np.mean(pc_)) / np.std(pc_)
        pc_ -= pc_.min(0, keepdims=1)
        
        _, inds, inverse_map = sparse_quantize(pc_,
                                               return_index=True,
                                               return_inverse=True)
        
        if len(inds) > self.num_points:
                inds = np.random.choice(inds, self.num_points, replace=False)
    
        pc = pc_[inds]
        labels = labels_[inds]
        
        
        feed_dict = {
            'pc_': pc_,
            'targets_': labels_,
            'file_name': self.files[index],
            'pc': pc,
            'targets': labels,
            'inds': inds,
            'inverse_map': inverse_map,
            'file_name': self.files[index]
        }
        return feed_dict
    
    def get_point_voxel(self, block, labels_, index):
        pc_ = np.round(block[:, :3] / self.voxel_size).astype(np.int32)
        pc_ -= pc_.min(0, keepdims=1)

        feat_ = block

        _, inds, inverse_map = sparse_quantize(pc_,
                                               return_index=True,
                                               return_inverse=True)

        if 'train' in self.split:
            if len(inds) > self.num_points:
                inds = np.random.choice(inds, self.num_points, replace=False)

        pc = pc_[inds]
        feat = feat_[inds]
        labels = labels_[inds]
        lidar = SparseTensor(feat, pc)
        labels = SparseTensor(labels, pc)
        labels_ = SparseTensor(labels_, pc_)
        inverse_map = SparseTensor(inverse_map, pc_)

        if self.visualize == False:
            feed_dict = {
            'lidar': lidar,
            'targets': labels,
            'targets_mapped': labels_,
            'inverse_map': inverse_map,
            'file_name': self.files[index]
        }

        else:
            feed_dict = {
                'pc': block[:, :3],
                'label': labels_,
                'lidar': lidar,
                'targets': labels,
                'targets_mapped': labels_,
                'inverse_map': inverse_map,
                'file_name': self.files[index]
            }

        return feed_dict

In [103]:
root = "/ws/data/erasor_carla/carla_dataset"
voxel_size = 0.1
num_points = 300000
visualize = True
sample_stride = 1
split = "train"
submit = True
window = 10

In [104]:
dataset = ErasorCarlaInternal(root, voxel_size, num_points, visualize, split, submit)

In [105]:
map_data, scan_data = next(iter(dataset))

In [106]:
scan_data.keys()

dict_keys(['pc_', 'targets_', 'file_name', 'pc', 'targets', 'inds', 'inverse_map'])

In [107]:
# np.shape(scan_data['pc_'])
# np.shape(scan_data['pc'][scan_data['inverse_map']])

In [108]:
points_ = scan_data['pc']
labels = scan_data['targets']
orig_points = scan_data['pc'][scan_data['inverse_map']]
orig_labels = scan_data['targets'][scan_data['inverse_map']]

In [109]:
points = (points_ - points_.mean()) / points_.std()

In [110]:
label2color = {0: (1.0, 0.0, 0.0), 1: (0.0, 0.0, 1.0), 2:(0.0, 0.0, 1.0)}
colors = np.array([map_color[x] for x in labels])

In [111]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)
visualizer = JVisualizer()
visualizer.add_geometry(pcd)
visualizer.show()

JVisualizer with 1 geometries

In [112]:
np.shape(points)

(211160, 3)

In [102]:
mpoints_ = map_data['pc']
mlabels = map_data['targets']
# orig_points = map_data['pc'][map_data['inverse_map']]
# orig_labels = map_data['targets'][map_data['inverse_map']]

In [87]:
mpoints = (mpoints_ - mpoints_.mean()) / mpoints_.std()

In [88]:
label2color = {0: (1.0, 0.0, 0.0), 1: (0.0, 0.0, 1.0), 2:(0.0, 0.0, 1.0)}
mcolors = np.array([map_color[x] for x in mlabels])

In [89]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(mpoints)
pcd.colors = o3d.utility.Vector3dVector(mcolors)
visualizer = JVisualizer()
visualizer.add_geometry(pcd)
visualizer.show()

JVisualizer with 1 geometries

In [91]:
np.shape(mpoints)

(300000, 3)

In [90]:
sspoints_ = np.mgrid[1:3:0.2, 2:6:0.2, 0:2:0.2]
sspoints = sspoints_.reshape(3, -1).T
sslabels = np.ones([np.shape(sspoints)[0], 1])

In [44]:
points

array([[-1.19546345,  0.59915196, -1.19161234],
       [-1.18391013,  0.59530085, -1.19161234],
       [-1.17235681,  0.58759864, -1.19161234],
       ...,
       [ 1.97399681, -0.34822012, -1.01831257],
       [ 1.99710345, -0.29045353, -1.01831257],
       [ 1.99710345, -0.28660243, -1.01831257]])

In [183]:
np.shape(sspoints)

(2000, 3)

In [184]:
np.shape(sslabels)

(2000, 1)

In [185]:
ssblock = np.concatenate((sspoints, sslabels), axis=1)

In [162]:
points = np.concatenate((points, sspoints), axis=0)

In [164]:
points

array([[-1.18889817,  0.44287013, -1.18523127],
       [-1.18889817,  0.45387082, -1.18523127],
       [-1.18523127,  0.39886739, -1.18523127],
       ...,
       [ 5.8       ,  4.8       ,  1.4       ],
       [ 5.8       ,  4.8       ,  1.6       ],
       [ 5.8       ,  4.8       ,  1.8       ]])